In [1]:
# general imports
import mdtraj as md
import msmbuilder
import nglview as nv
from msmbuilder.featurizer import LigandRMSDFeaturizer, RawPositionsFeaturizer
from msmbuilder.cluster import KMeans, KMedoids, KCenters
import numpy as np 
from collections import Counter
from msmbuilder.cluster import RegularSpatial

/Users/albaness/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/albaness/miniconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/albaness/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas

## RoA6 Analysis 

In [2]:
trajectory = md.load('/Users/albaness/github/musashi-new/yank/experiments/experiment-harmonic/Roasix/RoAsix_state0.pdb')

In [3]:
atoms_to_align = trajectory.top.select('backbone and resn !=UNK')
atoms_to_align

array([   0,    4,   17,   18,   19,   21,   39,   40,   41,   43,   56,
         57,   58,   60,   76,   77,   78,   80,   95,   96,   97,   99,
        102,  103,  104,  106,  109,  110,  111,  113,  128,  129,  130,
        132,  139,  140,  141,  143,  163,  164,  165,  167,  180,  181,
        182,  184,  194,  195,  196,  198,  205,  206,  207,  217,  219,
        220,  221,  223,  231,  232,  233,  235,  242,  243,  244,  246,
        261,  262,  263,  265,  285,  286,  287,  289,  297,  298,  299,
        301,  318,  319,  320,  322,  338,  339,  340,  342,  349,  350,
        351,  353,  371,  372,  373,  375,  391,  392,  393,  395,  398,
        399,  400,  402,  413,  414,  415,  417,  432,  433,  434,  436,
        456,  457,  458,  460,  471,  472,  473,  475,  482,  483,  484,
        486,  499,  500,  501,  503,  515,  516,  517,  519,  532,  533,
        534,  536,  556,  557,  558,  560,  568,  569,  570,  580,  582,
        583,  584,  586,  596,  597,  598,  600,  6

In [4]:
aligned_traj = trajectory.superpose(trajectory[0], frame=0, atom_indices=atoms_to_align)

In [5]:
ligand = aligned_traj.topology.select('resn UNK')
ligand

array([1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353,
       1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364,
       1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,
       1376, 1377])

In [6]:
ligand_trajectory = aligned_traj.atom_slice(ligand)

In [7]:
len(trajectory)

1676

In [8]:
reg_space = RegularSpatial(d_min=0.1, metric='rmsd').fit(ligand_trajectory)

In [9]:
for i,center in enumerate(reg_space.cluster_center_indices_):
    aligned_traj[center][0].save('musashi_ro6_h_reg_center_cluster%s_frame%s.pdb' % (i,center[0]))

In [10]:
reg_list = reg_space.fit_predict(ligand_trajectory)

In [11]:
converted_labels = []
for x in reg_list:
    converted_labels.append(x[0])
Counter(converted_labels).most_common

<bound method Counter.most_common of Counter({183: 502, 126: 308, 21: 243, 82: 167, 239: 89, 176: 50, 124: 45, 6: 42, 76: 40, 214: 29, 152: 26, 55: 18, 145: 18, 241: 15, 259: 10, 115: 8, 0: 7, 148: 7, 140: 7, 23: 4, 1: 3, 20: 3, 37: 3, 98: 3, 252: 3, 3: 2, 129: 2, 146: 2, 266: 2, 2: 1, 17: 1, 36: 1, 49: 1, 51: 1, 52: 1, 72: 1, 77: 1, 92: 1, 181: 1, 117: 1, 122: 1, 135: 1, 136: 1, 138: 1, 223: 1, 32: 1, 267: 1})>

In [12]:
nsamples = 10
for x in [183, 126, 21]:
    indices_in_cluster = []
    for i,y in enumerate(converted_labels):
        if y == x: 
            indices_in_cluster.append(i)
    samples = np.random.choice(indices_in_cluster, nsamples, replace=False)
    for frame in samples:  
        aligned_traj[frame].save('musashi_ro6_h_reg_cluster%s_frame%s.pdb' % (x,frame))
        
aligned_traj[frame].save('musashi_ro6_h_reg_cluster%s_frame%s.pdb' % (37, 99))
aligned_traj[frame].save('musashi_ro6_h_reg_cluster%s_frame%s.pdb' % (17, 35))
aligned_traj[frame].save('musashi_ro6_h_reg_cluster%s_frame%s.pdb' % (201, 1190))

In [13]:
len(reg_list)

1676

# RoOH analysis

In [15]:
trajectory = md.load('/Users/albaness/github/musashi-new/yank/experiments/experiment-harmonic/RoOH/Rooh_h_state0.h5')

In [16]:
atoms_to_align = trajectory.top.select('backbone and resn !=UNK')
atoms_to_align

array([   0,    4,   17,   18,   19,   21,   39,   40,   41,   43,   56,
         57,   58,   60,   76,   77,   78,   80,   95,   96,   97,   99,
        102,  103,  104,  106,  109,  110,  111,  113,  128,  129,  130,
        132,  139,  140,  141,  143,  163,  164,  165,  167,  180,  181,
        182,  184,  194,  195,  196,  198,  205,  206,  207,  217,  219,
        220,  221,  223,  231,  232,  233,  235,  242,  243,  244,  246,
        261,  262,  263,  265,  285,  286,  287,  289,  297,  298,  299,
        301,  318,  319,  320,  322,  338,  339,  340,  342,  349,  350,
        351,  353,  371,  372,  373,  375,  391,  392,  393,  395,  398,
        399,  400,  402,  413,  414,  415,  417,  432,  433,  434,  436,
        456,  457,  458,  460,  471,  472,  473,  475,  482,  483,  484,
        486,  499,  500,  501,  503,  515,  516,  517,  519,  532,  533,
        534,  536,  556,  557,  558,  560,  568,  569,  570,  580,  582,
        583,  584,  586,  596,  597,  598,  600,  6

In [17]:
aligned_traj = trajectory.superpose(trajectory[0], frame=0, atom_indices=atoms_to_align)

In [18]:
ligand = aligned_traj.topology.select('resn UNK')
ligand
ligand_trajectory = aligned_traj.atom_slice(ligand)
ligand

array([1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353,
       1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364,
       1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374])

In [19]:
reg_space = RegularSpatial(d_min=0.10, metric='rmsd').fit(ligand_trajectory)

In [20]:
for i,center in enumerate(reg_space.cluster_center_indices_):
    aligned_traj[center][0].save('musashi_rooh_h_reg_center_cluster%s_frame%s.pdb' % (i,center[0]))

In [21]:
reg_list = reg_space.partial_predict(ligand_trajectory)

In [22]:
converted_labels = []
for x in reg_list:
    converted_labels.append(x)
converted_labels[:10]
Counter(converted_labels)

Counter({0: 1502,
         1: 37,
         2: 228,
         3: 1090,
         4: 736,
         5: 487,
         6: 467,
         7: 470,
         8: 101,
         9: 79,
         10: 42,
         11: 93,
         13: 95,
         14: 853,
         15: 23,
         16: 389,
         17: 235,
         18: 349,
         19: 315,
         20: 214,
         21: 1,
         28: 46,
         29: 104,
         30: 1,
         31: 59,
         33: 1,
         35: 10,
         36: 31,
         37: 13,
         41: 219,
         43: 53,
         44: 158})

In [23]:
reg_space.cluster_center_indices_

array([[   0,    0],
       [   3,    0],
       [   4,    0],
       [   6,    0],
       [   7,    0],
       [  18,    0],
       [  20,    0],
       [  51,    0],
       [  76,    0],
       [  97,    0],
       [ 109,    0],
       [ 133,    0],
       [ 188,    0],
       [ 308,    0],
       [ 414,    0],
       [ 691,    0],
       [ 752,    0],
       [ 926,    0],
       [1000,    0],
       [1049,    0],
       [1151,    0],
       [1378,    0],
       [1483,    0],
       [1809,    0],
       [2060,    0],
       [2069,    0],
       [2492,    0],
       [2692,    0],
       [2813,    0],
       [3069,    0],
       [3119,    0],
       [3367,    0],
       [3625,    0],
       [3954,    0],
       [4298,    0],
       [4388,    0],
       [4691,    0],
       [5363,    0],
       [5488,    0],
       [5914,    0],
       [6126,    0],
       [6616,    0],
       [6732,    0],
       [6861,    0],
       [7495,    0],
       [7509,    0]])

In [24]:
nsamples = 10
for x in [0, 3, 4, 14]:
    indices_in_cluster = []
    for i,y in enumerate(converted_labels):
        if y == x: 
            indices_in_cluster.append(i)
    samples = np.random.choice(indices_in_cluster, nsamples, replace=False)
    for frame in samples:  
        aligned_traj[frame].save('musashi_rooh_h_reg_cluster%s_frame%s.pdb' % (x,frame))

## Ro 08-2750 Analysis

In [35]:
trajectory = md.load('/Users/albaness/github/musashi-new/yank/experiments/experiment-harmonic/Roeight/Roeight_h_state0.h5')

In [36]:
atoms_to_align = trajectory.top.select('backbone and resn !=UNK')
atoms_to_align

array([   0,    4,   17,   18,   19,   21,   39,   40,   41,   43,   56,
         57,   58,   60,   76,   77,   78,   80,   95,   96,   97,   99,
        102,  103,  104,  106,  109,  110,  111,  113,  128,  129,  130,
        132,  139,  140,  141,  143,  163,  164,  165,  167,  180,  181,
        182,  184,  194,  195,  196,  198,  205,  206,  207,  217,  219,
        220,  221,  223,  231,  232,  233,  235,  242,  243,  244,  246,
        261,  262,  263,  265,  285,  286,  287,  289,  297,  298,  299,
        301,  318,  319,  320,  322,  338,  339,  340,  342,  349,  350,
        351,  353,  371,  372,  373,  375,  391,  392,  393,  395,  398,
        399,  400,  402,  413,  414,  415,  417,  432,  433,  434,  436,
        456,  457,  458,  460,  471,  472,  473,  475,  482,  483,  484,
        486,  499,  500,  501,  503,  515,  516,  517,  519,  532,  533,
        534,  536,  556,  557,  558,  560,  568,  569,  570,  580,  582,
        583,  584,  586,  596,  597,  598,  600,  6

In [37]:
aligned_traj = trajectory.superpose(trajectory[0], frame=0, atom_indices=atoms_to_align)

In [38]:
ligand = aligned_traj.topology.select('resn UNK')
ligand
ligand_trajectory = aligned_traj.atom_slice(ligand)
ligand

array([1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353,
       1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364,
       1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372])

In [39]:
reg_space = RegularSpatial(d_min=0.10, metric='rmsd').fit(ligand_trajectory)

In [40]:
for i,center in enumerate(reg_space.cluster_center_indices_):
    aligned_traj[center][0].save('musashi_roeight_h_reg_center_cluster%s_frame%s.pdb' % (i,center[0]))

In [41]:
reg_list = reg_space.partial_predict(ligand_trajectory)

In [42]:
converted_labels = []
for x in reg_list:
    converted_labels.append(x)
converted_labels[:10]
Counter(converted_labels)

Counter({0: 2189,
         1: 2743,
         2: 11,
         3: 2,
         6: 2958,
         7: 219,
         9: 42,
         10: 28,
         11: 273,
         13: 1,
         14: 21,
         15: 6,
         16: 8})

In [43]:
reg_space.cluster_center_indices_

array([[   0,    0],
       [   2,    0],
       [   6,    0],
       [  74,    0],
       [ 120,    0],
       [ 221,    0],
       [ 535,    0],
       [ 755,    0],
       [ 893,    0],
       [1803,    0],
       [1913,    0],
       [2554,    0],
       [3082,    0],
       [5813,    0],
       [7632,    0],
       [7749,    0],
       [8315,    0]])

In [44]:
nsamples = 10
for x in [0, 1, 6]:
    indices_in_cluster = []
    for i,y in enumerate(converted_labels):
        if y == x: 
            indices_in_cluster.append(i)
    samples = np.random.choice(indices_in_cluster, nsamples, replace=False)
    for frame in samples:  
        aligned_traj[frame].save('musashi_roeight_h_reg_cluster%s_frame%s.pdb' % (x,frame))